# 15.PPO（Proximal Policy Optimization）：近端策略优化
> **Proximal Policy Optimization (PPO) 算法** 在 2017 年由 John Schulman 等人提出：[Proximal Policy Optimization Algorithms](https://arxiv.org/abs/1707.06347)
> **TRPO** 计算的复杂性较高，尤其是在需要计算海森矩阵和执行二阶优化时，导致其不适用于计算资源有限的情况。**PPO** 基于 **TRPO**  的思想，但是其算法实现更加简单。并且大量的实验结果表明，与 **TRPO**  相比，**PPO** 能学习得一样好（甚至更快），这使得 **PPO** 成为非常流行的强化学习算法。

## 15.1 PPO 算法原理
> **TRPO** 使用 **泰勒展开近似、共轭梯度、线性搜索等** 方法直接求解：
$$\max_{\theta} \mathbb{E}_{s\sim\nu^{\pi_{\theta_k}}}\mathbb{E}_{a\sim\pi_{\theta_k}(\cdot|s)}\left[\frac{\pi_{\theta^{\prime}}(a|s)}{\pi_{\theta_k}(a|s)}A^{\pi_{\theta_k}}(s,a)\right]$$
$$\mathbb{E}_{s\sim\nu^{\pi_{\theta_k}}}[D_{KL}(\pi_{\theta_k}(\cdot|s),\pi_{\theta^{\prime}}(\cdot|s))]\leq \delta$$

- **PPO** 采用了相对简单且高效的 ***方法*** 来实现这一目标，避免了 **TRPO** 中计算海森矩阵和执行二阶优化的复杂性。
- ***方法*** 主要有两种形式：**Clipped Surrogate Objective (剪切目标函数) 、 Adaptive KL Penalty (自适应 KL 惩罚)** 

### Clipped Surrogate Objective (剪切目标函数)
> 这种形式是 **PPO** 最常用的方式，因为在大多数任务中能够提供较好的效果，并且实现起来更加简单和高效
> 它通过引入 ***剪切机制*** 来约束策略更新的幅度：
$$\arg\max_{\theta}\mathbb{E}_{s\sim\nu^{\pi_{\theta_k}}}\mathbb{E}_{a\sim\pi_{\theta_k}(\cdot|s)}\left[\min\left(\frac{\pi_\theta(a|s)}{\pi_{\theta_k}(a|s)}A^{\pi_{\theta_k}}(s,a) , \operatorname{clip}\left(\frac{\pi_\theta(a|s)}{\pi_{\theta_k}(a|s)},1-\epsilon,1+\epsilon\right)A^{\pi_{\theta_k}}(s,a)\right)\right]$$

- $\epsilon$ 是一个小的常数（例如0.1或0.2）,控制剪切的范围
- $\operatorname{clip}(x,l,r)$：把 $x$ 限制在 $[l,r]$ 内

> $A^{\pi_{\theta_k}}(s,a)>0$，说明这个动作的价值高于平均，最大化这个式子会增大$\frac{\pi_\theta(\alpha|s)}{\pi_{\theta_k}(a|s)}$，但不会让其超过$1+\epsilon$；
> $A^{\pi_{\theta_k}}(s,a)<0$，最大化这个式子会减小$\frac{\pi_\theta(\alpha|s)}{\pi_{\theta_k}(a|s)}$，但不会让其超过$1-\epsilon$。**以此避免策略更新过大，保持更好的稳定性**


> 所以 **PPO** 避免了使用 **KL 散度约束**（不需要计算复杂的二阶导数或海森矩阵），而是使用 **剪切机制（Clipping）** 来替代。目标函数变为：
$$L^{\mathrm{clip}}(\theta)=\mathbb{E}_t\left[\min\left(r_t(\theta)\hat{A}_t,\mathrm{clip}(r_t(\theta),1-\epsilon,1+\epsilon)\hat{A}_t\right)\right]$$

- $r_t(\theta):$ 是当前策略和旧策略的概率比率：

$$r_t(\theta)=\frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_\mathrm{old}}(a_t|s_t)}$$
- 剪切目标函数通过限制$r_t(\theta)$的范围来 **控制策略更新**

### Adaptive KL Penalty (自适应 KL 惩罚)

## 15.2 PPO 代码实践
> 仍在 **车杆、倒立摆** 两个环境中测试 **PPO 算法**
> 大量实验表明，**PPO-Clip** 总是比 **PPO-Penalty** 表现得更好。因此下面专注于 **PPO-Clip** 的代码实现: